In [2]:
!pip install lxml

In [3]:
import pandas as pd
from pathlib import Path

Project_Root = Path(".")
Data_Dir = Project_Root / "sports_data"

excel_files = sorted(Data_Dir.glob("*.xls*"))
excel_files

[WindowsPath('sports_data/sportsref_download_2010-2011.xls'),
 WindowsPath('sports_data/sportsref_download_2011-2012.xls'),
 WindowsPath('sports_data/sportsref_download_2012-2013.xls'),
 WindowsPath('sports_data/sportsref_download_2013-2014.xls'),
 WindowsPath('sports_data/sportsref_download_2014-2015.xls'),
 WindowsPath('sports_data/sportsref_download_2015-2016.xls'),
 WindowsPath('sports_data/sportsref_download_2016-2017.xls'),
 WindowsPath('sports_data/sportsref_download_2017-2018.xls'),
 WindowsPath('sports_data/sportsref_download_2018-2019.xls'),
 WindowsPath('sports_data/sportsref_download_2019-2020.xls'),
 WindowsPath('sports_data/sportsref_download_2020-2021.xls'),
 WindowsPath('sports_data/sportsref_download_2021-2022.xls'),
 WindowsPath('sports_data/sportsref_download_2022-2023.xlsx'),
 WindowsPath('sports_data/sportsref_download_2023-2024.xlsx'),
 WindowsPath('sports_data/sportsref_download_2024-2025.xlsx')]

In [4]:
def clean_season(path, season_name):
    """
    Load and clean one season file from sports_data.
    Handles:
    - .xls files that are actually HTML tables (Sports Reference)
    - .xlsx real Excel files with two header rows
    """

    ext = path.suffix.lower()

    # ---- Step 1: read the raw table with a 2-row header ----
    if ext == ".xls":
        # Sports Reference "xls" exports are HTML tables
        tables = pd.read_html(path, header=[0, 1])
        raw = tables[0]
    elif ext == ".xlsx":
        raw = pd.read_excel(path, header=[0, 1], engine="openpyxl")
    else:
        raise ValueError(f"Unsupported file type: {ext}")

    # ---- Step 2: flatten the 2-level column names ----
    new_cols = []
    for col in raw.columns:
        # col is usually a tuple: (top, sub)
        if isinstance(col, tuple):
            top = str(col[0])
            sub = str(col[1])
        else:
            top = ""
            sub = str(col)

        # Special cases
        if top.startswith("Unnamed") and sub == "School":
            name = "School"
        elif top == "Overall":
            # We care about overall G, W, L, etc.
            name = sub
        elif top == "Totals":
            # We care about totals FG, 3P, TRB, etc.
            name = sub
        elif top == "Points" and sub == "Tm.":
            name = "PTS"
        elif top == "Points" and sub == "Opp.":
            name = "Opp PTS"
        else:
            # For Conf., Home, Away, etc., create a unique name
            # that won't match keep_cols (so we ignore them)
            name = f"{top}_{sub}".strip()

        new_cols.append(name)

    raw.columns = new_cols

    # ---- Step 3: drop completely empty columns ----
    raw = raw.dropna(axis=1, how="all")

    # ---- Step 4: keep a limited set of stats (now matching real names) ----
    keep_cols = [
        "School",
        "G", "W", "L", "W-L%", "SRS", "SOS",
        "PTS", "Opp PTS",
        "FG", "FGA", "FG%",
        "3P", "3PA", "3P%",
        "FT", "FTA", "FT%",
        "TRB", "AST", "STL", "BLK", "TOV"
    ]

    existing_cols = [col for col in keep_cols if col in raw.columns]
    missing_cols = [col for col in keep_cols if col not in raw.columns]

    if missing_cols:
        print(f"For {season_name}, missing columns (skipped): {missing_cols}")

    df = raw[existing_cols].copy()

    # ---- Step 4b: ensure no duplicate column names remain ----
    df = df.loc[:, ~df.columns.duplicated()]

    # ---- Step 5: clean School names ----
    if "School" in df.columns:
        df["School"] = (
            df["School"]
            .astype(str)
            .str.replace(r"\s*NCAA$", "", regex=True)
            .str.replace(r"\s*\(.*?\)", "", regex=True)  # remove seeds like (1)
            .str.strip()
        )

    # ---- Step 6: convert numeric columns ----
    for col in df.columns:
        if col != "School":
            # Ensure we're operating on a Series, not a DataFrame
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # ---- Step 7: add Season column ----
    df["Season"] = season_name

    return df

In [5]:
season_dfs = {}

for file in excel_files:
    name = file.stem
    season_name = name.split("_")[-1]  # e.g. "2010-2011"

    df_season = clean_season(file, season_name)
    season_dfs[season_name] = df_season

    print("Loaded season:", season_name, "Shape:", df_season.shape)


Loaded season: 2010-2011 Shape: (345, 24)
Loaded season: 2011-2012 Shape: (344, 24)
Loaded season: 2012-2013 Shape: (347, 24)
Loaded season: 2013-2014 Shape: (351, 24)
Loaded season: 2014-2015 Shape: (351, 24)
Loaded season: 2015-2016 Shape: (351, 24)
Loaded season: 2016-2017 Shape: (351, 24)
Loaded season: 2017-2018 Shape: (351, 24)
Loaded season: 2018-2019 Shape: (353, 24)
Loaded season: 2019-2020 Shape: (353, 24)
Loaded season: 2020-2021 Shape: (347, 24)
Loaded season: 2021-2022 Shape: (358, 24)
Loaded season: 2022-2023 Shape: (363, 24)
Loaded season: 2023-2024 Shape: (363, 24)
Loaded season: 2024-2025 Shape: (364, 24)


In [6]:
!pip install openpyxl


In [7]:
tables = pd.read_html(excel_files[0])
tables[0].columns.tolist()

[('Unnamed: 0_level_0', 'Rk'),
 ('Unnamed: 1_level_0', 'School'),
 ('Overall', 'G'),
 ('Overall', 'W'),
 ('Overall', 'L'),
 ('Overall', 'W-L%'),
 ('Overall', 'SRS'),
 ('Overall', 'SOS'),
 ('Unnamed: 8_level_0', 'Unnamed: 8_level_1'),
 ('Conf.', 'W'),
 ('Conf.', 'L'),
 ('Unnamed: 11_level_0', 'Unnamed: 11_level_1'),
 ('Home', 'W'),
 ('Home', 'L'),
 ('Unnamed: 14_level_0', 'Unnamed: 14_level_1'),
 ('Away', 'W'),
 ('Away', 'L'),
 ('Unnamed: 17_level_0', 'Unnamed: 17_level_1'),
 ('Points', 'Tm.'),
 ('Points', 'Opp.'),
 ('Unnamed: 20_level_0', 'Unnamed: 20_level_1'),
 ('Totals', 'MP'),
 ('Totals', 'FG'),
 ('Totals', 'FGA'),
 ('Totals', 'FG%'),
 ('Totals', '3P'),
 ('Totals', '3PA'),
 ('Totals', '3P%'),
 ('Totals', 'FT'),
 ('Totals', 'FTA'),
 ('Totals', 'FT%'),
 ('Totals', 'ORB'),
 ('Totals', 'TRB'),
 ('Totals', 'AST'),
 ('Totals', 'STL'),
 ('Totals', 'BLK'),
 ('Totals', 'TOV'),
 ('Totals', 'PF')]

In [8]:
pd.read_excel(excel_files[12], header=0, engine="openpyxl").columns.tolist()

['Unnamed: 0',
 'Unnamed: 1',
 'Overall',
 'Unnamed: 3',
 'Unnamed: 4',
 'Unnamed: 5',
 'Unnamed: 6',
 'Unnamed: 7',
 'Unnamed: 8',
 'Conf.',
 'Unnamed: 10',
 'Unnamed: 11',
 'Home',
 'Unnamed: 13',
 'Unnamed: 14',
 'Away',
 'Unnamed: 16',
 'Unnamed: 17',
 'Points',
 'Unnamed: 19',
 'Unnamed: 20',
 'Totals',
 'Unnamed: 22',
 'Unnamed: 23',
 'Unnamed: 24',
 'Unnamed: 25',
 'Unnamed: 26',
 'Unnamed: 27',
 'Unnamed: 28',
 'Unnamed: 29',
 'Unnamed: 30',
 'Unnamed: 31',
 'Unnamed: 32',
 'Unnamed: 33',
 'Unnamed: 34',
 'Unnamed: 35',
 'Unnamed: 36',
 'Unnamed: 37']

In [17]:
df_all = pd.concat(season_dfs.values(), ignore_index=True)
df_all.info()
df_all.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5292 entries, 0 to 5291
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   School   5292 non-null   object 
 1   G        5291 non-null   float64
 2   W        5291 non-null   float64
 3   L        5291 non-null   float64
 4   W-L%     5291 non-null   float64
 5   SRS      5291 non-null   float64
 6   SOS      5291 non-null   float64
 7   PTS      5291 non-null   float64
 8   Opp PTS  5291 non-null   float64
 9   FG       5291 non-null   float64
 10  FGA      5291 non-null   float64
 11  FG%      5291 non-null   float64
 12  3P       5291 non-null   float64
 13  3PA      5291 non-null   float64
 14  3P%      5291 non-null   float64
 15  FT       5291 non-null   float64
 16  FTA      5291 non-null   float64
 17  FT%      5291 non-null   float64
 18  TRB      5291 non-null   float64
 19  AST      5291 non-null   float64
 20  STL      5291 non-null   float64
 21  BLK      5291 

,School,G,W,L,W-L%,SRS,SOS,PTS,Opp PTS,FG,...,3P%,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,Season
0,Air Force,32.0,16.0,16.0,0.500,2.71,3.31,2074.0,2082.0,722.0,...,0.377,418.0,593.0,0.705,918.0,494.0,200.0,83.0,386.0,2010-2011
1,Akron,36.0,23.0,13.0,0.639,2.15,-1.02,2524.0,2365.0,892.0,...,0.361,457.0,649.0,0.704,1238.0,520.0,240.0,129.0,432.0,2010-2011
2,Alabama,37.0,25.0,12.0,0.676,11.70,4.27,2473.0,2198.0,930.0,...,0.298,461.0,652.0,0.707,1339.0,454.0,351.0,190.0,516.0,2010-2011
3,Alabama A&M,28.0,13.0,15.0,0.464,-15.19,-11.75,1808.0,1870.0,633.0,...,0.288,417.0,668.0,0.624,1058.0,339.0,233.0,151.0,444.0,2010-2011
4,Alabama State,35.0,17.0,18.0,0.486,-13.37,-10.31,2146.0,2234.0,728.0,...,0.283,525.0,870.0,0.603,1271.0,404.0,255.0,163.0,551.0,2010-2011
5,Albany,32.0,16.0,16.0,0.500,-7.05,-5.96,2013.0,2048.0,724.0,...,0.378,327.0,507.0,0.645,1155.0,370.0,150.0,90.0,411.0,2010-2011
6,Alcorn State,28.0,4.0,24.0,0.143,-23.02,-8.02,1838.0,2258.0,614.0,...,0.323,494.0,781.0,0.633,992.0,255.0,218.0,103.0,473.0,2010-2011
7,American,31.0,22.0,9.0,0.710,-3.13,-5.64,2051.0,1973.0,716.0,...,0.338,431.0,600.0,0.718,1044.0,430.0,127.0,86.0,346.0,2010-2011
8,Appalachian State,31.0,16.0,15.0,0.516,-4.81,-2.95,2265.0,2269.0,790.0,...,0.337,489.0,694.0,0.705,1067.0,327.0,179.0,89.0,403.0,2010-2011
9,Arizona,38.0,30.0,8.0,0.789,16.54,7.99,2905.0,2580.0,969.0,...,0.397,671.0,899.0,0.746,1319.0,536.0,197.0,98.0,484.0,2010-2011


In [18]:
df_all.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5292 entries, 0 to 5291
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   School   5292 non-null   object 
 1   G        5291 non-null   float64
 2   W        5291 non-null   float64
 3   L        5291 non-null   float64
 4   W-L%     5291 non-null   float64
 5   SRS      5291 non-null   float64
 6   SOS      5291 non-null   float64
 7   PTS      5291 non-null   float64
 8   Opp PTS  5291 non-null   float64
 9   FG       5291 non-null   float64
 10  FGA      5291 non-null   float64
 11  FG%      5291 non-null   float64
 12  3P       5291 non-null   float64
 13  3PA      5291 non-null   float64
 14  3P%      5291 non-null   float64
 15  FT       5291 non-null   float64
 16  FTA      5291 non-null   float64
 17  FT%      5291 non-null   float64
 18  TRB      5291 non-null   float64
 19  AST      5291 non-null   float64
 20  STL      5291 non-null   float64
 21  BLK      5291 